In [1]:

import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
import seaborn as sns
#import xgboost as xgb
sns.set_style('whitegrid') 
%matplotlib inline

import xgboost as xgb
from sklearn import pipeline, grid_search

import random
random.seed(2016)

import time

/usr/local/lib/python2.7/dist-packages/scikit_learn-0.18.1-py2.7-linux-x86_64.egg/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/scikit_learn-0.18.1-py2.7-linux-x86_64.egg/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [5]:
def main(input1='../data/preprocessing_train_df.csv', input2='../data/preprocessing_test_df.csv'):



    start_time = time.time() 
    
#load preprocessed training data as a dataframe
    train_df = pd.read_csv(input1, index_col=0)
    test_df  = pd.read_csv(input2, index_col=0)

#  test_df = pd.read_csv("../data/preprocessing_test_df.csv" , index_col=0)
#  train_df.head()
#    train_df.info()
#    test_df.info()
#    test_df.head(3)


    x_train = train_df.iloc[:, 2:].as_matrix()
    y_train = train_df.iloc[:, 1]

    id_test = test_df['PassengerId']
    x_test = test_df.iloc[:,1:].as_matrix()
#    x_train.info()
    
    
    print('--- Features Set: %s minutes ---' % round(((time.time() - start_time) / 60), 2))
#    print('Number of Features: ', len(x_train.columns.tolist()))
    
    
# Grid Search for XGBClassifier    
    
    
    clf = xgb.XGBClassifier(seed=2016)

    param_grid = {
        'n_estimators': [1000],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [3,4,5],
        'subsample': [0.8],
        'colsample_bytree': [0.8] 
    }
#    param_grid = {
#        'n_estimators': [300],
#        'learning_rate': [0.05],
#        'max_depth': [7],
#        'subsample': [0.8],
#        'colsample_bytree': [0.8, 0.85] 
#    }
        
    

    model = grid_search.GridSearchCV(estimator=clf, param_grid=param_grid, 
                                     scoring='accuracy', cv=5, n_jobs=1)
    model.fit(x_train, y_train)
    
    
    print('--- Grid Search Completed: %s minutes ---' % round(((time.time() - start_time) / 60), 2))
    print('Param grid:')
    print(param_grid)
    print('Best Params:')
    print(model.best_params_)
    print('Best CV Score:')
    print(-model.best_score_)

    
    y_pred = model.predict(x_test)

    pd.DataFrame({'PassengerId': id_test, 'Survived': y_pred}).to_csv('../data/submission_xgb.csv', index=False)
    
    print('--- Submission Generated: %s minutes ---' % round(((time.time() - start_time) / 60), 2))


#if __name__ == '__main__':


main()


--- Features Set: 0.0 minutes ---
--- Grid Search Completed: 0.17 minutes ---
Param grid:
{'n_estimators': [1000], 'subsample': [0.8], 'learning_rate': [0.2], 'colsample_bytree': [0.8], 'max_depth': [3, 4, 5]}
Best Params:
{'n_estimators': 1000, 'subsample': 0.8, 'learning_rate': 0.2, 'colsample_bytree': 0.8, 'max_depth': 5}
Best CV Score:
-0.821548821549
--- Submission Generated: 0.17 minutes ---
